Author: Rudi Kreidenhuber, <Rudi.Kreidenhuber@gmail.com>, 
License: BSD (3-clause)

## To do:

Summary statistics (Number of occurences of Semio-Markers, EEG-Markers, ..., EKG?)

Figure out how to add plotly figs to mne.Report()


----

# Video EEG Monitoring Annotation visualizer

----


## Inputs:
 - .edf-files you wish to analyze go into ./data folder

## Run:
 - Press play :-)

## Outputs:
 - Found in results folder
 - Results for single files are put into a folder that matches the input-filename

----

## Howto:
 1. **Mark Events in EEG file using the following prefixes:**
 - e- --> EEG marker
 - s- --> Semiology marker
 - no prefix --> Everything else (clinical tests during/ after seizure)
 - i- --> Marker to ignore for focused analysis

 - One marker **must (!) contain "Beginn"** --> this is considered the seizure onset (if it is missing, onset is set to zero)
 - every marker **can** contain Beginn, for example:
 - Onset first seen in EEG --> Markername "e-asdBeginnfgh" --> would still be recognized as EEG marker and seizure onset
 2. **Save EEG file in .edf format and copy to ./data folder**
 - Every file in this folder is going to be analyzed, if it ends with .edf
 

----
## Configuration
----

### Parameters:
graph_sep_line_width
- How far blue dashed seperator lines are apart from each other

plot_tmin
- First time point in seconds from onset, that should be included in the visualization, set to 0 to deactivate

plot_tmax
- Last time point in seconds from onset, that should be included in the visualization, set to 0 to deactivate


In [ ]:
subj_name = "Patient"
graph_sep_line_width = 5
plot_tmin = -20
plot_tmax = 100

----
## Pipeline start
----

In [ ]:
# import everything
import os
import glob
import mne
from mne import Report
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import mpld3
import plotly.graph_objects as go
from utils import (get_parent_dir, extract_lab_sec, loc_of_sep_lines, plot_seizure_horizontal, plot_seizure_vertical,
                                        raw_to_df, extract_groups, extract_ordered_groups, save_plotly_to_html,
                                        shrink_df_to_tmax, create_results_folders, save_fig_to_disc, plot_interactive_subplot_with_table,
                                        extract_parameters_from_raw, plot_eventcounts)

# no need to show figures here
plt.ioff() 

# grab .edfs
edfs = glob.glob("../data/*.edf")

report = Report(subject=subj_name, title="Event summary")
pattern = "*.edf"
report.parse_folder("../data/", pattern=pattern)

----
## Static visualization
----

In [ ]:
if len(edfs) > 0:
    # Create folder structure
    create_results_folders(edfs)
    df = dict() 
    shrinked_df = dict()
    for e in edfs:
        print(f"Now processing file: {e}")
        raw = mne.io.read_raw(e, preload=True)                      #read
        df[e], onset = raw_to_df(raw, e)                            # annotations to DataFrame
        e_events, s_events, t_events = extract_ordered_groups(df[e], e.split("/")[-1])     # Extract groups

        # save
        source = e.split("/")[-1].split(".")[0]
        save_path = ("../results/" + source + "/" + "tables/All_events.tsv")
        df[e].to_csv(save_path, sep="\t")
        save_path = ("../results/" + source + "/" + "tables/EEG_events.tsv")
        e_events.to_csv(save_path, sep="\t")
        save_path = ("../results/" + source + "/" + "tables/Semiology_events.tsv")
        s_events.to_csv(save_path, sep="\t")
        save_path = ("../results/" + source + "/" + "tables/Test_events.tsv")
        t_events.to_csv(save_path, sep="\t")
                
        # Shrink df to tmax
        if df[e].iloc[0,0]:
            print(f"\nCalculating parameters for focused visualization, tmin = {plot_tmin}, tmax = {plot_tmax} s.\n\n\n")
            
            shrinked_df[e] = shrink_df_to_tmax(df=df[e], tmax=plot_tmax, tmin=plot_tmin)
            se_events, ss_events, st_events = extract_groups(shrinked_df[e], e)

            # Vsualizations
            seizure_vertical_w_limits = plot_seizure_vertical(df=shrinked_df[e], eeg=se_events, 
                                                                semio=ss_events, testing=st_events, 
                                                                source=e.split("/")[-1], tmin= plot_tmin, 
                                                                tmax=plot_tmax, name="seizure vertical with time limits",
                                                                graph_sep_line_width=graph_sep_line_width)
            cap = source + " --> seizure_vertical_w_limits"
            sec = source + "-shrinked"
            report.add_figs_to_section(seizure_vertical_w_limits, captions=cap, section=sec)
            save_fig_to_disc(seizure_vertical_w_limits, e, cap)

            seizure_horizontal_w_limits = plot_seizure_horizontal(df=shrinked_df[e], eeg=se_events, 
                                                                semio=ss_events, testing=st_events, 
                                                                source=e.split("/")[-1], tmin= plot_tmin, 
                                                                tmax=plot_tmax, name="seizure horizontal with time limits",
                                                                graph_sep_line_width=graph_sep_line_width)
            cap = source + " --> seizure_horizontal_w_limits"
            sec = source + "-shrinked"
            report.add_figs_to_section(seizure_horizontal_w_limits, captions=cap, section=sec)
            save_fig_to_disc(seizure_horizontal_w_limits, e, "seizure_horizontal_w_limits")

            event_counts = plot_eventcounts(df=shrinked_df[e], eeg=se_events, semio=ss_events, 
                                                                source=e.split("/")[-1])
            cap = source + " --> event_conuts_w_limits"
            sec = source + "-shrinked"
            report.add_figs_to_section(event_counts, captions=cap, section=sec)
            save_fig_to_disc(event_counts, e, "event_conuts_w_limits")


        seizure_vertical = plot_seizure_vertical(df=df[e], eeg=e_events, semio=s_events, 
                                                    testing=t_events, source=e.split("/")[-1], 
                                                    name="seizure vertical",
                                                    graph_sep_line_width=graph_sep_line_width)
        cap = source + " --> seizure_vertical"
        sec = source
        report.add_figs_to_section(seizure_vertical, captions=cap, section=sec)
        save_fig_to_disc(seizure_vertical, e, "seizure_vertical")

        seizure_horizontal = plot_seizure_horizontal(df=df[e], eeg=e_events, semio=s_events, 
                                                    testing=t_events, source=e.split("/")[-1], 
                                                    name="seizure_horizontal",
                                                    graph_sep_line_width=graph_sep_line_width)
        cap = source + " --> seizure_horizontal"
        sec = source
        report.add_figs_to_section(seizure_horizontal, captions=cap, section=sec)
        save_fig_to_disc(seizure_horizontal, e, "seizure_horizontal")

        event_counts = plot_eventcounts(df=df[e], eeg=e_events, semio=s_events, source=e.split("/")[-1])
        cap = source + " --> event_conuts"
        sec = source
        report.add_figs_to_section(event_counts, captions=cap, section=sec)
        save_fig_to_disc(event_counts, e, "event_conuts")

----
## Save data
----

In [ ]:
df_keys = df.keys()
# horizontal grand average
for idx, val in enumerate(df_keys):
    if idx == 0:
        concat = df[val]
        concat["source"] = val
        cols = list(concat)
        if not "source" in concat:
            cols.insert(0, cols.pop(cols.index('source')))
        concat = concat.loc[:, cols]
        print(concat.time_from_onset)
        concat = concat.sort_values(by=["time_from_onset"])
        concat = concat.drop(columns=["onset"], axis=1)
        concat["order_of_occurence"] = (1 + np.arange(len(concat.loc[:,"time_from_onset"])))
        
    if idx > 0:
        new_df = df[val]
        if not "source" in new_df.keys():
            new_df["source"] = val
        cols = list(new_df)
        cols.insert(0, cols.pop(cols.index('source')))
        new_df = new_df.loc[:, cols]
        new_df.drop(columns=["onset"], axis=1, inplace=True)
        new_df["order_of_occurence"] = (1 + np.arange(len(new_df.loc[:,"time_from_onset"]))).astype(int)
        concat = pd.merge(concat, new_df, how="outer", on="description", suffixes=(" ", "  "))
    #print(f"\n\n\n\nRun {idx} --> concat = {concat}")
    idx += 1

concat.to_csv("../results/grand_average//tables/All_Data_horizontal.tsv", sep="\t")


# EEG/Semio/Test horizontal grand_average
for idx, val in enumerate(df_keys):
    if idx == 0:
        eeg_ga, semio_ga, test_ga = extract_ordered_groups(df[val], source = val)
        # this should not be necessary:
        eeg_ga.sort_values(by=["time_from_onset"])
        semio_ga.sort_values(by=["time_from_onset"])
        test_ga.sort_values(by=["time_from_onset"])
        
    if idx > 0:
        new_df = df[val]
        ne, ns, nt = extract_ordered_groups(new_df, source = val)
        #new_df.insert(loc=0, column='source', value=val.split("/")[-1])
        eeg_ga = pd.merge(eeg_ga, ne, how="outer", on="description", suffixes=(" ", "  ")) 
        semio_ga = pd.merge(semio_ga, ns, how="outer", on="description", suffixes=(" ", "  "))
        test_ga = pd.merge(test_ga, nt, how="outer", on="description", suffixes=(" ", "  "))
    idx += 1

eeg_ga.drop(columns=["onset"], axis=1)
semio_ga.drop(columns=["onset"], axis=1)
test_ga.drop(columns=["onset"], axis=1)

eeg_ga = eeg_ga.sort_values(by=["time_from_onset"])
semio_ga = semio_ga.sort_values(by=["time_from_onset"])
test_ga = test_ga.sort_values(by=["time_from_onset"])

eeg_ga.to_csv("../results/grand_average//tables/EEE_grand_average.tsv", sep="\t")
semio_ga.to_csv("../results/grand_average//tables/Semiology_grand_average.tsv", sep="\t")
test_ga.to_csv("../results/grand_average//tables/Testing_grand_average.tsv", sep="\t")


# Vertical grand average
for idx, val in enumerate(df_keys):
    if idx == 0:
        concat = df[val]
        concat.loc[:, "source"] = val.split("/")[-1]        
    if idx > 0:
        new_df = df[val]
        new_df.loc[:, "source"] = val.split("/")[-1]
        concat = pd.concat([concat, new_df], axis=0)     
    idx += 1
concat.drop(columns=["onset"], axis=1)
concat.drop(index=4, axis=1, inplace=True)

concat = concat.sort_values(by=["time_from_onset"])
concat.to_csv("../results/grand_average/tables/All_events.tsv", sep="\t")

----
# Interactive Visualization
----

In [ ]:
# Seizure subplots
event_folders = glob.glob("../results/*")
data = dict()
interactive_plots = dict()
for e in event_folders:
    source = e.split("/")[-1]
    tsv = e + "/tables/All_events.tsv"
    data[source] = pd.read_csv(tsv, sep="\t")
    EEG, Semio, Test = extract_ordered_groups(data[source], source)
    interactive_plots[source] = plot_interactive_subplot_with_table(data[source], EEG, Semio, Test, title=source)
    save_plotly_to_html(interactive_plots[source], source=source)


In [ ]:
source = "grand_average"
html = "/Users/idrael/git/VEEG_Event_Processor/results/grand_average/viz/grand_average_interactive_viz.html"
report.add_htmls_to_section(html, section="test", captions="test")

In [ ]:
fig = interactive_plots["grand_average"]
html = mpld3.fig_to_html(fig)
report.add_figs_to_section(html, section="test", captions="test")

In [ ]:
report_save_name = "../results/cumulative_report.html"
report.save(report_save_name, overwrite=True)